<a href="https://colab.research.google.com/github/simon-mellergaard/GAI-with-LLMs/blob/main/Project%20codes/Assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
> *Evaluate HuggingFaceTB/SmolLM-135M-Instruct on the cnn_dailymail dataset used in Chapter 6 in Natural Language Processing with Transformers.*
> 1. *Compare four different experiments each with a different prompting and decoding strategy. Argue for why these are the most promising ones to try. Evaluate the rouge scores of each model on the test set (if compute becomes a problem use a smaller subset of the test set) and compare to the performance of PEGASUS.*

## Setup


In [1]:
# Install packages
!pip install datasets==3.6.0 # To load the cnn dataset
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nv

In [30]:
# Libraries
import evaluate
import os
import torch
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize

# Functions
from transformers import pipeline
from google.colab import userdata
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from huggingface_hub import login as login_hf
from wandb import login as login_wandb
from tqdm import tqdm

In [3]:
# Logging in to Huggingface and wandb
os.environ['HF_TOKEN'] = userdata.get('HF')
os.environ['WANDB_TOKEN'] = userdata.get('wandb')
login_hf(os.environ['HF_TOKEN'])
login_wandb(key = os.environ['WANDB_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: simonmellergaard (simonmellergaard-aarhus-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
# Setting up the device (GPU). Code is taken from the genaibook-module.
def get_device(cuda_ordinal=None):
    if torch.cuda.is_available():
        return torch.device("cuda", cuda_ordinal)
    if torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

device = get_device()

## Loading the dataset



In [7]:
# Loading the datasets
from datasets import load_dataset

dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

README.md: 0.00B [00:00, ?B/s]

cnn_dailymail.py: 0.00B [00:00, ?B/s]

The repository for ccdv/cnn_dailymail contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/cnn_dailymail.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features: ['article', 'highlights', 'id']


In [ ]:
# Sampling from the test set
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000))

## Test

In [109]:
model_name = "HuggingFaceTB/SmolLM-135M-Instruct" # Use gpt2 if run out of memory
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
sample_text = dataset["train"][1]["article"][:2000]

In [108]:
max_length = 128
input_txt = dataset["train"][1]["article"][:2000] + '\n\nTL;DR:\n'
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
torch.manual_seed(42)
output_greedy = model.generate(input_ids, max_length=max_length,
                               do_sample=True, top_p=0.95, top_k=10, temperature=0.4)
print(tokenizer.decode(output_greedy[0]))

Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never seriou

In [116]:
max_length = 128
input_txt = "Give a summary of the following text:\n\n" + sample_text + "\n\n Summary: "
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
torch.manual_seed(42)
output_greedy = model.generate(input_ids, max_length=max_length,
                               do_sample=True, top_p=0.8, top_k=50, temperature=0.1)
print(tokenizer.decode(output_greedy[0]))

Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Give a summary of the following text:

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016

## SMolLM

The following [SmolLM-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM-135M-Instruct) model is loaded:

In [59]:
sample_text = dataset["train"][1]["article"][:2000]
summaries = {}
queries = ['\nTL;DR:\n',
           '\nSummary of the text:']
for query in queries:
    pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM-135M-Instruct")
    smollm_query = sample_text + query
    pipe_out = pipe(smollm_query, max_length=512, clean_up_tokenization_spaces=True)
    summaries[query] = "\n".join(
        sent_tokenize(pipe_out[0]["generated_text"][len(smollm_query) :]))

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by p

In [129]:
    pipe = pipeline("summarization", model="HuggingFaceTB/SmolLM-135M-Instruct")

Device set to use cuda:0
The model 'LlamaForCausalLM' is not supported for summarization. Supported models are ['PeftModelForSeq2SeqLM', 'BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'GraniteSpeechForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForC

In [142]:
smollm_query = 'sumarize the following text: ' + sample_text + '\n Summary: '
pipe_out = pipe(smollm_query, max_length=50, clean_up_tokenization_spaces=True, do_sample=False, num_beams=5, no_repeat_ngram_size=4)
test_output = "\n".join(
    sent_tokenize(pipe_out[0]["summary_text"][len(smollm_query) :]))

Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [143]:
test_output

"2023-2024 Olympic Games: The United States and the United Kingdom will compete in the 26th and 27th editions of the Olympic Games, respectively.\nThe United States will participate in the 40th edition of the Games, which will be held in Tokyo, Japan.\nThe United Kingdom will participate in both the 21st and 22nd editions of the Olympics, which will take place in Rio de Janeiro, Brazil.\nThe US and the UK will compete in both the men's and women's events, with the United States winning the men's gold medal and the UK winning the women's gold medal.\nThe US will also participate in the women's singles event, while the UK will participate in men's singles and pair events.\nThe US Olympic Committee will also host the 23rd edition of the Olympics in New York City, which is scheduled to take place in September.\nThe United Nations General Assembly will also hold the 24th edition of this year's Olympics, which is expected to be held in New York, New York, from September 20 to September 24.\n

In [72]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


TL;DR:

The UK is facing an unprecedented problem with the UK's food supply.
The government is proposing a "Food Safety Bill" which would require food manufacturers to ensure that their products are safe to eat.
The UK is facing an unprecedented problem with the UK's food supply.
The government is proposing a "Food Safety Bill" which would require food manufacturers to ensure that their products are safe to eat.
The bill is seen as an opportunity to drive down food prices and improve food safety standards.
The concept of a "Food Safety Bill" is based on the idea that the government is committed to reducing food waste and improving food safety by requiring food manufacturers to test and meet certain standards.
The bill would require food manufacturers to:
* Ensure that their products meet certa

In [52]:
summaries

{'SmolLM': 'The 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe 10th of December, 2017.\nThe'}

In [15]:
# Loading the rouge metric
rouge_metric = evaluate.load("rouge")

In [25]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries_smollm(dataset, metric, model, tokenizer,
                               batch_size=8, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [31]:
model_ckpt = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_smollm(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=2) # REDUCED BATCH SIZE

  0%|          | 0/500 [00:00<?, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  0%|          | 1/500 [00:03<25:59,  3.13s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Both `max_new_tokens` (=40) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  0%|          | 2/500 [00:06<25:44,  3.10s/it]A decoder-only architec

In [35]:
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["SmolLM"])

,rouge1,rouge2,rougeL,rougeLsum
SmolLM,0.144227,0.082236,0.105634,0.126501


## Comparing with PEGASUS

In [ ]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=2) # REDUCED BATCH SIZE